# Using the abstract Keras backend to write new code

If you want the Keras modules you write to be compatible with both Theano (th) and TensorFlow (tf), you have to write them via the abstract Keras backend API. Here's an intro.

You can import the backend module via:

In [1]:
import numpy as np
#np.random.seed(5)   #not useful in Keras
import tensorflow as tf
#tf.random.set_random_seed(3)
from keras import backend as K

Using TensorFlow backend.



The code below instantiates an input placeholder. It's equivalent to tf.placeholder() or th.tensor.matrix(), th.tensor.tensor3(), etc.

In [2]:
inputs = K.placeholder(shape=(2,4,5))
# also works:
inputs = K.placeholder(shape=(None,4,5))
# also works:
inputs = K.placeholder(ndim=3)

The code below instantiates a variable. It's equivalent to tf.Variable() or th.shared().

In [3]:
val = np.random.random((3,4,5))
val2 = np.random.random((3,4,5))
var = K.variable(value=val)

In [4]:
output = var + inputs

In [5]:
tf_session = K.get_session()
output.eval(session=tf_session, feed_dict = {inputs : val2})
var.eval(session=tf_session)

array([[[0.66060865, 0.0300301 , 0.99725485, 0.24316595, 0.5119618 ],
        [0.107789  , 0.78278834, 0.98394746, 0.19380465, 0.435569  ],
        [0.64456475, 0.9110368 , 0.17065303, 0.40758392, 0.49796164],
        [0.87327564, 0.16381188, 0.6642294 , 0.40012708, 0.8198158 ]],

       [[0.17371033, 0.6957717 , 0.04525151, 0.25430807, 0.05457584],
        [0.6423691 , 0.95979714, 0.9533505 , 0.2982089 , 0.7182005 ],
        [0.9070376 , 0.89414877, 0.4168126 , 0.3779643 , 0.9929056 ],
        [0.2706096 , 0.56470174, 0.41066158, 0.24349687, 0.2560679 ]],

       [[0.8207083 , 0.4083376 , 0.3186262 , 0.50797236, 0.49546114],
        [0.5404506 , 0.29895705, 0.31473577, 0.4738359 , 0.07929014],
        [0.99618226, 0.91186774, 0.7770949 , 0.26252526, 0.83046305],
        [0.18783174, 0.8193949 , 0.34181574, 0.65593415, 0.09306999]]],
      dtype=float32)

Most tensor operations you will need can be done as you would in TensorFlow or Theano:

In [6]:
# Initializing Tensors with Random Numbers
b = K.random_uniform_variable(shape=(4,3), low=0,high=1, seed=3) # Uniform distribution
c = K.random_normal_variable(shape=(2,3), mean=0, scale=10, seed=3)  # Gaussian distribution

# Tensor Arithmetic
a = K.dot(b , K.transpose(c))
s = K.sum(a , axis = 1)
soft = K.softmax(s)
concat = K.concatenate([soft, s], axis = 0)
tf_session = K.get_session()
K.shape(concat).eval(session=tf_session)    #Keras shape
#concat.eval(session=tf_session)
#soft.eval(session=tf_session)
#s.eval(session=tf_session)
#b.eval(session=tf_session)
#c.eval(session=tf_session)

array([8], dtype=int32)



# Backend functions

backend : Publicly accessible method for determining the current backend.

In [7]:
K.backend() #keras.backend.backend()

'tensorflow'

epsilon : Returns the value of the fuzz factor used in numeric expressions.

In [8]:
K.epsilon()  #keras.backend.epsilon()

1e-07

set_epsilon : Sets the value of the fuzz factor used in numeric expressions.

In [9]:
K.set_epsilon(1e-08)  #keras.backend.set_epsilon(e)

In [10]:
K.epsilon()  #keras.backend.epsilon()

1e-08

floatx: Returns the default float type, as a string. (e.g. 'float16', 'float32', 'float64').

In [11]:
K.floatx()   #keras.backend.floatx()

'float32'

image_data_format : Returns the default image data format convention.

In [12]:
K.image_data_format()  #keras.backend.image_data_format()

'channels_last'

clear_session : Destroys the current TF graph and creates a new one.  Useful to avoid clutter from old models / layers.

In [13]:
K.clear_session() #keras.backend.clear_session() 

learning_phase : Returns the learning phase flag. The learning phase flag is a bool tensor (0 = test, 1 = train) to be passed as input to any Keras function that uses a different behavior at train time and test time.

In [14]:
s1 = K.get_session()
K.learning_phase().eval(session=s1) #keras.backend.learning_phase()

False

set_learning_phase : Sets the learning phase to a fixed value. 

In [15]:
K.set_learning_phase(True)   #keras.backend.set_learning_phase()
K.learning_phase()

True

update : Update the value of x to new_x.

In [16]:
x1 = tf.Variable(tf.random.normal((2,3)), name = 'x1')
new_x = tf.random.normal((2,3), mean=0, stddev=10, seed=2)
K.update(x1, new_x).eval(session=s1)

array([[ -8.581108 ,  -1.9662298,   1.3895046],
       [-12.212768 ,  -4.0341287, -11.454041 ]], dtype=float32)

moving_average_update : Compute the moving average of a variable.

In [27]:
#keras.backend.moving_average_update(x, value, momentum)
m = tf.Variable(tf.random.normal((2,3)), name = 'm')
val_m = tf.random.normal((2,3), mean=0, stddev=10, seed=3)
K.moving_average_update(m, val_m, momentum=0.9)
s2 = K.get_session()
m.eval(session=s2)

array([[ 0.405457  , -1.1697458 , -1.5269387 ],
       [ 0.14035547, -0.9657406 , -1.0671937 ]], dtype=float32)

dot :  Multiplies 2 tensors (and/or variables) and returns a tensor. When attempting to multiply a nD tensor with a nD tensor, it reproduces the Theano behavior. (e.g.  (2, 3) * (4, 3, 5) -> (2, 4, 5)) . Returns A tensor, dot product of x and y.



In [34]:
#keras.backend.dot(x, y)
l = tf.random_normal((3,4),  mean=50, stddev=100, seed=4)
n = tf.random_normal((2,4,5), mean=0, stddev=1, seed=4)
o = K.dot(l, n)
K.shape(o).eval(session=s2)

array([3, 2, 5], dtype=int32)